In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# CNN pour la classification d'images - MNIST\n",
        "\n",
        "## BTS SIO  - Séance 2: Types de réseaux de neurones\n",
        "\n",
        "Ce notebook vous guidera à travers l'implémentation et l'utilisation d'un réseau de neurones convolutif (CNN) pour la classification d'images, en utilisant le célèbre dataset MNIST des chiffres manuscrits.\n",
        "\n",
        "### Objectifs d'apprentissage:\n",
        "- Comprendre l'architecture d'un réseau convolutif (CNN)\n",
        "- Implémenter un CNN avec TensorFlow/Keras\n",
        "- Visualiser les filtres et feature maps\n",
        "- Analyser les performances du modèle\n",
        "\n",
        "### Prérequis:\n",
        "- Connaissances de base en Python\n",
        "- Avoir suivi la séance 1 d'introduction au Deep Learning"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 1. Configuration de l'environnement\n",
        "\n",
        "Commençons par importer les bibliothèques nécessaires."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import tensorflow as tf\n",
        "from tensorflow.keras.models import Sequential\n",
        "from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout\n",
        "from tensorflow.keras.utils import to_categorical\n",
        "from tensorflow.keras.datasets import mnist\n",
        "import time\n",
        "import seaborn as sns\n",
        "from sklearn.metrics import confusion_matrix\n",
        "\n",
        "# Configuration pour reproductibilité\n",
        "np.random.seed(42)\n",
        "tf.random.set_seed(42)\n",
        "\n",
        "# Vérifier la version de TensorFlow\n",
        "print(f\"TensorFlow version: {tf.__version__}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 2. Chargement et préparation du dataset MNIST\n",
        "\n",
        "Le dataset MNIST contient 70,000 images de chiffres manuscrits de taille 28x28 pixels."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "print(\"Chargement des données MNIST...\")\n",
        "(X_train, y_train), (X_test, y_test) = mnist.load_data()\n",
        "\n",
        "# Afficher les dimensions des données\n",
        "print(f\"Dimensions de X_train: {X_train.shape}\")\n",
        "print(f\"Dimensions de y_train: {y_train.shape}\")\n",
        "print(f\"Dimensions de X_test: {X_test.shape}\")\n",
        "print(f\"Dimensions de y_test: {y_test.shape}\")\n",
        "print(f\"Nombre de classes: {len(np.unique(y_train))}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Préparation des données pour le CNN\n",
        "\n",
        "Pour utiliser nos images avec un CNN, nous devons :\n",
        "1. Ajouter une dimension pour le canal (les images sont en niveaux de gris, donc 1 seul canal)\n",
        "2. Normaliser les valeurs de pixels entre 0 et 1\n",
        "3. Convertir les étiquettes en format one-hot encoding"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Redimensionnement et normalisation\n",
        "X_train = X_train.reshape(-1, 28, 28, 1) / 255.0\n",
        "X_test = X_test.reshape(-1, 28, 28, 1) / 255.0\n",
        "\n",
        "# Conversion des étiquettes en catégories one-hot\n",
        "y_train_onehot = to_categorical(y_train, 10)\n",
        "y_test_onehot = to_categorical(y_test, 10)\n",
        "\n",
        "print(f\"Nouvelle forme de X_train: {X_train.shape}\")\n",
        "print(f\"Nouvelle forme de y_train_onehot: {y_train_onehot.shape}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation de quelques exemples\n",
        "\n",
        "Regardons à quoi ressemblent nos données."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "plt.figure(figsize=(10, 5))\n",
        "for i in range(10):\n",
        "    plt.subplot(2, 5, i+1)\n",
        "    plt.imshow(X_train[i].reshape(28, 28), cmap='gray')\n",
        "    plt.title(f\"Chiffre: {y_train[i]}\")\n",
        "    plt.axis('off')\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3. Création d'un modèle CNN\n",
        "\n",
        "Un CNN est un type de réseau de neurones spécialisé pour traiter des données ayant une structure en grille, comme les images. Les principales couches sont :\n",
        "\n",
        "1. **Couches de convolution (Conv2D)** : Détectent des caractéristiques locales (lignes, formes...)\n",
        "2. **Couches de pooling (MaxPooling2D)** : Réduisent la dimension des données\n",
        "3. **Couches denses (Dense)** : Effectuent la classification finale\n",
        "\n",
        "Nous allons créer un CNN simple avec 2 couches de convolution pour classifier les chiffres MNIST."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Créer un modèle CNN\n",
        "model = Sequential([\n",
        "    # Première couche de convolution\n",
        "    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv1'),\n",
        "    MaxPooling2D((2, 2), name='pool1'),\n",
        "    \n",
        "    # Deuxième couche de convolution\n",
        "    Conv2D(64, (3, 3), activation='relu', name='conv2'),\n",
        "    MaxPooling2D((2, 2), name='pool2'),\n",
        "    \n",
        "    # Aplatissement pour passer aux couches denses\n",
        "    Flatten(name='flatten'),\n",
        "    \n",
        "    # Couches denses (fully connected)\n",
        "    Dense(128, activation='relu', name='dense1'),\n",
        "    Dropout(0.5, name='dropout1'),  # Évite le surapprentissage\n",
        "    Dense(10, activation='softmax', name='output')  # 10 classes (chiffres 0-9)\n",
        "])\n",
        "\n",
        "# Compiler le modèle\n",
        "model.compile(\n",
        "    optimizer='adam',\n",
        "    loss='categorical_crossentropy',\n",
        "    metrics=['accuracy']\n",
        ")\n",
        "\n",
        "# Afficher le résumé de l'architecture\n",
        "model.summary()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4. Entraînement du modèle\n",
        "\n",
        "Entraînons maintenant notre CNN sur les données MNIST."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Entraînement du modèle\n",
        "start_time = time.time()\n",
        "\n",
        "history = model.fit(\n",
        "    X_train, \n",
        "    y_train_onehot, \n",
        "    batch_size=128, \n",
        "    epochs=5,  # Nombre réduit d'époques pour la démonstration\n",
        "    validation_split=0.2,  # 20% des données d'entraînement pour la validation\n",
        "    verbose=1\n",
        ")\n",
        "\n",
        "training_time = time.time() - start_time\n",
        "print(f\"\\nTemps d'entraînement: {training_time:.2f} secondes\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation de l'évolution de l'entraînement"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "plt.figure(figsize=(12, 4))\n",
        "\n",
        "# Graphique de précision\n",
        "plt.subplot(1, 2, 1)\n",
        "plt.plot(history.history['accuracy'], label='Entraînement')\n",
        "plt.plot(history.history['val_accuracy'], label='Validation')\n",
        "plt.title('Évolution de la précision')\n",
        "plt.xlabel('Époque')\n",
        "plt.ylabel('Précision')\n",
        "plt.legend()\n",
        "plt.grid(True, linestyle='--', alpha=0.6)\n",
        "\n",
        "# Graphique de perte\n",
        "plt.subplot(1, 2, 2)\n",
        "plt.plot(history.history['loss'], label='Entraînement')\n",
        "plt.plot(history.history['val_loss'], label='Validation')\n",
        "plt.title('Évolution de la perte')\n",
        "plt.xlabel('Époque')\n",
        "plt.ylabel('Perte')\n",
        "plt.legend()\n",
        "plt.grid(True, linestyle='--', alpha=0.6)\n",
        "\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5. Évaluation du modèle\n",
        "\n",
        "Évaluons notre modèle sur l'ensemble de test."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Évaluation sur l'ensemble de test\n",
        "test_loss, test_acc = model.evaluate(X_test, y_test_onehot, verbose=1)\n",
        "print(f\"Précision sur l'ensemble de test: {test_acc*100:.2f}%\")\n",
        "\n",
        "# Prédictions\n",
        "y_pred = model.predict(X_test)\n",
        "y_pred_classes = np.argmax(y_pred, axis=1)\n",
        "\n",
        "# Matrice de confusion\n",
        "conf_mat = confusion_matrix(y_test, y_pred_classes)\n",
        "plt.figure(figsize=(10, 8))\n",
        "sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')\n",
        "plt.xlabel('Prédit')\n",
        "plt.ylabel('Réel')\n",
        "plt.title('Matrice de confusion')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation des exemples mal classifiés\n",
        "\n",
        "Explorons quelques exemples que notre modèle a mal classifiés."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Identifier les erreurs\n",
        "misclassified_indices = np.where(y_pred_classes != y_test)[0]\n",
        "misclassified_count = len(misclassified_indices)\n",
        "print(f\"Nombre total d'erreurs: {misclassified_count} sur {len(y_test)} images de test\")\n",
        "\n",
        "# Afficher quelques exemples mal classifiés\n",
        "num_examples = min(10, misclassified_count)\n",
        "plt.figure(figsize=(15, 6))\n",
        "\n",
        "for i, idx in enumerate(misclassified_indices[:num_examples]):\n",
        "    plt.subplot(2, 5, i+1)\n",
        "    plt.imshow(X_test[idx].reshape(28, 28), cmap='gray')\n",
        "    plt.title(f\"Réel: {y_test[idx]}\\nPrédit: {y_pred_classes[idx]}\")\n",
        "    plt.axis('off')\n",
        "    \n",
        "plt.tight_layout()\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### 🧠 Réflexion sur les erreurs\n",
        "\n",
        "**Question**: En observant les exemples mal classifiés, quelles pourraient être les raisons de ces erreurs? Notez vos observations et hypothèses ci-dessous.\n",
        "\n",
        "**Points à considérer:**\n",
        "- Certains chiffres sont-ils plus souvent confondus que d'autres?\n",
        "- Quelles caractéristiques visuelles communes peuvent expliquer les erreurs?"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "*Écrivez vos observations ici...*"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 6. Visualisation des filtres et feature maps\n",
        "\n",
        "Une des grandes forces des CNNs est leur interprétabilité visuelle. Explorons ce que le réseau \"voit\" réellement."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Fonction pour visualiser les filtres de convolution\n",
        "def visualize_filters(model, layer_name, num_filters=8):\n",
        "    \"\"\"Visualise les filtres d'une couche de convolution\"\"\"\n",
        "    \n",
        "    # Récupérer les poids du filtre de la couche spécifiée\n",
        "    filters, biases = model.get_layer(layer_name).get_weights()\n",
        "    \n",
        "    # Normaliser les filtres pour une meilleure visualisation\n",
        "    f_min, f_max = filters.min(), filters.max()\n",
        "    filters = (filters - f_min) / (f_max - f_min)\n",
        "    \n",
        "    # Afficher les premiers filtres\n",
        "    plt.figure(figsize=(12, 4))\n",
        "    for i in range(num_filters):\n",
        "        plt.subplot(2, 4, i+1)\n",
        "        # Pour la première couche de convolution, les filtres sont 3D (hauteur, largeur, canaux)\n",
        "        # Nous affichons le filtre pour le premier canal (0)\n",
        "        plt.imshow(filters[:, :, 0, i], cmap='viridis')\n",
        "        plt.title(f'Filtre {i+1}')\n",
        "        plt.axis('off')\n",
        "    plt.suptitle(f'Filtres de la couche {layer_name}')\n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "\n",
        "# Visualiser les filtres de la première couche de convolution\n",
        "visualize_filters(model, 'conv1')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation des feature maps (cartes d'activation)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def visualize_feature_maps(model, image, layer_name, num_features=8):\n",
        "    \"\"\"Visualise les feature maps (activations) d'une couche pour une image donnée\"\"\"\n",
        "    \n",
        "    # Créer un modèle qui renvoie les activations de la couche spécifiée\n",
        "    layer_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)\n",
        "    \n",
        "    # Obtenir les activations pour une image\n",
        "    feature_maps = layer_model.predict(image.reshape(1, 28, 28, 1))\n",
        "    \n",
        "    # Afficher les premières cartes d'activation\n",
        "    plt.figure(figsize=(12, 4))\n",
        "    for i in range(min(num_features, feature_maps.shape[3])):\n",
        "        plt.subplot(2, 4, i+1)\n",
        "        plt.imshow(feature_maps[0, :, :, i], cmap='viridis')\n",
        "        plt.title(f'Feature {i+1}')\n",
        "        plt.axis('off')\n",
        "    plt.suptitle(f'Feature Maps de la couche {layer_name}')\n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "\n",
        "# Choisir une image de test\n",
        "sample_idx = 12  # Vous pouvez essayer avec différents indices\n",
        "sample_image = X_test[sample_idx]\n",
        "\n",
        "# Afficher l'image originale\n",
        "plt.figure(figsize=(3, 3))\n",
        "plt.imshow(sample_image.reshape(28, 28), cmap='gray')\n",
        "plt.title(f\"Chiffre: {y_test[sample_idx]}\")\n",
        "plt.axis('off')\n",
        "plt.show()\n",
        "\n",
        "# Visualiser les feature maps pour chaque couche de convolution\n",
        "print(\"Feature maps de la première couche de convolution:\")\n",
        "visualize_feature_maps(model, sample_image, 'conv1')\n",
        "\n",
        "print(\"Feature maps de la deuxième couche de convolution:\")\n",
        "visualize_feature_maps(model, sample_image, 'conv2')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### 💡 Interprétation des feature maps\n",
        "\n",
        "Les feature maps nous montrent ce que \"voit\" chaque filtre de convolution :\n",
        "\n",
        "- **Première couche** : Détecte principalement des caractéristiques de base comme les bords et les contours\n",
        "- **Deuxième couche** : Combine ces caractéristiques de base pour détecter des formes plus complexes\n",
        "\n",
        "Cette hiérarchie de représentations est ce qui rend les CNNs si puissants pour la vision par ordinateur."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 7. Test avec des images bruitées\n",
        "\n",
        "Testons la robustesse de notre modèle face à des perturbations."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Fonction pour ajouter du bruit aux images\n",
        "def add_noise(images, noise_level=0.2):\n",
        "    \"\"\"Ajoute du bruit gaussien aux images\"\"\"\n",
        "    noisy_images = images.copy()\n",
        "    noise = np.random.normal(0, noise_level, images.shape)\n",
        "    noisy_images = noisy_images + noise\n",
        "    # Assurer que les valeurs restent entre 0 et 1\n",
        "    return np.clip(noisy_images, 0, 1)\n",
        "\n",
        "# Créer des versions bruitées de quelques images de test\n",
        "num_test_images = 5\n",
        "test_samples = X_test[:num_test_images]\n",
        "noisy_samples = add_noise(test_samples, noise_level=0.3)\n",
        "\n",
        "# Visualiser les images originales et bruitées\n",
        "plt.figure(figsize=(12, 4))\n",
        "for i in range(num_test_images):\n",
        "    # Image originale\n",
        "    plt.subplot(2, num_test_images, i+1)\n",
        "    plt.imshow(test_samples[i].reshape(28, 28), cmap='gray')\n",
        "    plt.title(f\"Original: {y_test[i]}\")\n",
        "    plt.axis('off')\n",
        "    \n",
        "    # Image bruitée\n",
        "    plt.subplot(2, num_test_images, i+num_test_images+1)\n",
        "    plt.imshow(noisy_samples[i].reshape(28, 28), cmap='gray')\n",
        "    plt.axis('off')\n",
        "    \n",
        "plt.tight_layout()\n",
        "plt.show()\n",
        "\n",
        "# Prédire sur les images bruitées\n",
        "noisy_predictions = model.predict(noisy_samples)\n",
        "noisy_pred_classes = np.argmax(noisy_predictions, axis=1)\n",
        "\n",
        "# Afficher les résultats\n",
        "print(\"Résultats des prédictions sur les images bruitées:\")\n",
        "for i in range(num_test_images):\n",
        "    status = \"✓\" if noisy_pred_classes[i] == y_test[i] else \"✗\"\n",
        "    print(f\"Image {i+1} - Réel: {y_test[i]}, Prédit: {noisy_pred_classes[i]} {status}\")\n",
        "\n",
        "# Calculer la précision sur les images bruitées\n",
        "accuracy_on_noisy = np.mean(noisy_pred_classes == y_test[:num_test_images]) * 100\n",
        "print(f\"\\nPrécision sur les images bruitées: {accuracy_on_noisy:.1f}%\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 8. Exercice : Amélioration du modèle\n",
        "\n",
        "À vous de jouer ! Essayez de modifier l'architecture du modèle pour améliorer ses performances. Voici quelques suggestions :\n",
        "\n",
        "1. Ajouter plus de couches de convolution\n",
        "2. Modifier le nombre de filtres\n",
        "3. Changer la taille des filtres\n",
        "4. Ajuster les paramètres d'entraînement (epochs, batch_size)\n",
        "\n",
        "Copiez le code de création du modèle ci-dessous et modifiez-le :"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# VOTRE CODE ICI\n",
        "# Créez votre propre modèle amélioré\n",
        "\n",
        "improved_model = Sequential([\n",
        "    # Modifiez l'architecture ici\n",
        "    \n",
        "])\n",
        "\n",
        "# Compiler le modèle\n",
        "improved_model.compile(\n",
        "    optimizer='adam',\n",
        "    loss='categorical_crossentropy',\n",
        "    metrics=['accuracy']\n",
        ")\n",
        "\n",
        "# Afficher le résumé\n",
        "improved_model.summary()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Entraînez votre modèle amélioré\n",
        "# history = improved_model.fit(...)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 9. Conclusion\n",
        "\n",
        "Dans ce notebook, vous avez :\n",
        "- Créé et entraîné un réseau de neurones convolutif (CNN) pour la classification d'images\n",
        "- Visualisé les filtres et les feature maps pour comprendre ce que \"voit\" le réseau\n",
        "- Évalué les performances du modèle et sa robustesse face au bruit\n",
        "\n",
        "Les CNN sont la base de nombreuses applications modernes de vision par ordinateur comme la reconnaissance faciale, la détection d'objets, et bien d'autres."
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.8.5"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 4
}